In [4]:
import kotlin.random.Random
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*

fun flowSomething(): Flow<Int> = flow {
    repeat(10) {
        emit(Random.nextInt(0, 500))
        delay(10L)
    }
}

runBlocking {
    flowSomething().collect { println(it) }
}


122
483
486
266
329
462
311
210
267
420


In [5]:
fun flowSomething(): Flow<Int> = flow {
    repeat(10) {
        emit(Random.nextInt(0, 500))
        delay(100L)
    }
}

runBlocking<Unit> {
    val res = withTimeoutOrNull(500L) {
        flowSomething().collect {
            println(it)
        }
        true
    } ?: false

    if ( ! res ) {
        println("it's canceled!!")
    }
}

403
439
466
389
414
it's canceled!!


In [6]:
runBlocking<Unit> {
    flowOf(1, 2, 3, 4, 5).collect {
        println(it)
    }
}

1
2
3
4
5


In [7]:
runBlocking<Unit> {
    listOf(1, 2, 3, 4, 5).asFlow()
        .collect {
            println(it)
        }

    (6..10).asFlow().collect{
        println(it)
    }
}

1
2
3
4
5
6
7
8
9
10


In [8]:
fun flowSomething(): Flow<Int> = flow {
    repeat(10) {
        emit(Random.nextInt(0, 500))
        delay(10L)
    }
}

runBlocking {
    flowSomething().map {
        "$it $it"
    }.collect { println(it) }
}

61 61
99 99
440 440
419 419
333 333
490 490
161 161
68 68
451 451
312 312


In [9]:
runBlocking<Unit> {
    (1..20).asFlow().filter {
        (it % 2) == 0
    }.collect {
        println(it)
    }
}

2
4
6
8
10
12
14
16
18
20


In [10]:
runBlocking<Unit> {
    (1..20).asFlow().filterNot {
        (it % 2) == 0
    }.collect {
        println(it)
    }
}

1
3
5
7
9
11
13
15
17
19


In [11]:
suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    (1..20).asFlow().transform {
        emit(it)
        emit(someCalc(it))
    }.collect {
        println(it)
    }
}

1
2
2
4
3
6
4
8
5
10
6
12
7
14
8
16
9
18
10
20
11
22
12
24
13
26
14
28
15
30
16
32
17
34
18
36
19
38
20
40


In [12]:
suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    (1..20).asFlow().transform {
        emit(it)
        emit(someCalc(it))
    }.take(5).collect { println(it) }
}

1
2
2
4
3


In [13]:
suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    (1..20).asFlow().transform {
        emit(it)
        emit(someCalc(it))
    }.takeWhile {
        it < 15
    }.collect {
        println(it)
    }
}

1
2
2
4
3
6
4
8
5
10
6
12
7
14
8


In [14]:
suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    (1..20).asFlow().transform {
        emit(it)
        emit(someCalc(it))
    }.drop(5)
        .collect{
            println(it)
        }
}

6
4
8
5
10
6
12
7
14
8
16
9
18
10
20
11
22
12
24
13
26
14
28
15
30
16
32
17
34
18
36
19
38
20
40


In [15]:
suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    val value = (1..10).asFlow()
        .reduce { a, b ->
            a + b
        }

    println(value)
}

55


In [16]:
import kotlin.random.Random
import kotlinx.coroutines.*
import kotlinx.coroutines.flow.*


suspend fun someCalc(i: Int): Int {
    delay(10L)
    return i * 2
}

runBlocking<Unit> {
    val value = (1..10).asFlow()
        .fold(10) { a, b ->
            a + b
        }
    println(value)
}


65


In [17]:
runBlocking<Unit> {
    val counter = (1..10)
        .asFlow()
        .count{ it % 2 == 0 }
    println(counter)
}

5


In [18]:
fun log(msg: String) = println("[${Thread.currentThread().name}] $msg")

fun simple(): Flow<Int> = flow {
    log("let's start flow!")

    for (i in 1..10)
        emit(i)

}

runBlocking<Unit> {
    launch(Dispatchers.IO) {
        simple()
            .collect{
                log("$it 를 받음")
            }
    }
}

In [20]:
fun log(msg: String) = println("[${Thread.currentThread().name}] $msg")

fun simple(): Flow<Int> = flow {
    withContext(Dispatchers.Default) {
        for (i in 1..10) {
            delay(100L)
            emit(i)
        }
    }
}

runBlocking<Unit> {
    launch(Dispatchers.IO) {
        simple()
            .collect { value -> log("${value}를 받음.") }
    }
}

java.lang.IllegalStateException: Flow invariant is violated:
		Flow was collected in [StandaloneCoroutine{Active}@34a33fce, Dispatchers.IO],
		but emission happened in [DispatchedCoroutine{Active}@70b6de12, Dispatchers.Default].
		Please refer to 'flow' documentation or use 'flowOn' instead

In [21]:
fun log(msg: String) = println("[${Thread.currentThread().name}] $msg")

fun simple(): Flow<Int> = flow {
    for (i in 1..10) {
        delay(100L)
        log("값 $i 를 emit 합니다.")
        emit(i)
    }
}.flowOn(Dispatchers.Default)

runBlocking<Unit> {
    simple().collect {
        log("$it 을 받음")
    }
}

[Execution of code 'fun log(msg: String)...'] 1 을 받음
[Execution of code 'fun log(msg: String)...'] 2 을 받음
[Execution of code 'fun log(msg: String)...'] 3 을 받음
[Execution of code 'fun log(msg: String)...'] 4 을 받음
[Execution of code 'fun log(msg: String)...'] 5 을 받음
[Execution of code 'fun log(msg: String)...'] 6 을 받음
[Execution of code 'fun log(msg: String)...'] 7 을 받음
[Execution of code 'fun log(msg: String)...'] 8 을 받음
[Execution of code 'fun log(msg: String)...'] 9 을 받음
[Execution of code 'fun log(msg: String)...'] 10 을 받음
